<a href="https://colab.research.google.com/github/Tiioluwani/My-virtual-assistant/blob/main/House_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import sklearn.metrics


In [ ]:
! pip install comet-ml


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.8/586.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.7/514.7 kB 21.9 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.7.0
    Uninstalling websocket-client-1.7.0:
      Successfully uninstalled websocket-client-1.7.0
  Attempting uninstall: python-box
    Found existing installation: python-box 7.1.1
    Uninstalling python-box-7.1.1:
      Successfully uninstalled python-box-7.1.1


In [ ]:
from comet_ml import Experiment

In [ ]:
from google.colab import files
uploaded =files.upload()

Saving data.csv to data.csv


In [ ]:
import  io
data=pd.read_csv(io.BytesIO(uploaded['data.csv']))
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [ ]:
target = data['MEDV']
target

0      24.0
1      21.6
2      34.7
3      33.4
4      36.2
       ... 
506    12.0
507    32.0
508    54.0
509    67.0
510    24.0
Name: MEDV, Length: 511, dtype: float64

In [ ]:
X = data
y = target

In [ ]:
# Splitting dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:

# Check for NaN values in the dataset
nan_values = data.isnull().values.any()

if nan_values:
    print("The dataset contains NaN values.")
else:
    print("The dataset does not contain NaN values.")


The dataset contains NaN values.


In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# Dealing with a NaNs value using imputation method
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

# Create a pipeline with an imputer and a linear regression model
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # You can change strategy as needed
    ('model', LinearRegression())
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)


Pipeline(steps=[('imputer', SimpleImputer()), ('model', LinearRegression())])

In [ ]:
# We create a Comet.ML experiment and deploy the model
experiment = Experiment(api_key='', project_name ='House_price_prediction')

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/koded-ii/house-price-prediction/98e4826f29914eacaa7e3f14adb4dc25



In [ ]:
# Logging our data statistics
experiment.log_dataset_hash('data.csv')


In [ ]:
# Log hyperparameters and configurations
experiment.log_parameters({"learning_rate": 0.001, "batch_size": 32})

In [ ]:
# We define the model and its parameters
model_definition = {
    'model': pipeline,
    'hyperparameters': {'regularization': 0.01}
}

In [ ]:
from sklearn.metrics import r2_score

# Predict using the trained model
y_pred = pipeline.predict(X_test)  # Assuming X_test is your test data

# Calculate R-squared
r_squared = r2_score(y_test, y_pred)

# Log R-squared as a dictionary
experiment.log_metrics({"R-squared": r_squared})


In [ ]:
import joblib

In [ ]:
# Assuming 'imputer' and 'model' are fitted objects in your pipeline
imputer = pipeline.named_steps['imputer']
model = pipeline.named_steps['model']

In [ ]:
# Save the components as separate files using joblib or pickle
joblib.dump(imputer, 'imputer.joblib')
joblib.dump(model, 'model.joblib')

['model.joblib']

In [ ]:
# Log the saved components
experiment.log_asset('imputer.joblib')
experiment.log_asset('model.joblib')


{'web': 'https://www.comet.com/api/asset/download?assetId=4ac8d054ee8843f6abe2b0dc87c444a2&experimentKey=98e4826f29914eacaa7e3f14adb4dc25',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=4ac8d054ee8843f6abe2b0dc87c444a2&experimentKey=98e4826f29914eacaa7e3f14adb4dc25',
 'assetId': '4ac8d054ee8843f6abe2b0dc87c444a2'}

In [ ]:
experiment.set_name('House Price Prediction')

In [ ]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/koded-ii/house-price-prediction/98e4826f29914eacaa7e3f14adb4dc25
COMET INFO:   Metrics:
COMET INFO:     R-squared : 1.0
COMET INFO:   Others:
COMET INFO:     Name : House Price Prediction
COMET INFO:   Parameters:
COMET INFO:     batch_size    : 32
COMET INFO:     learning_rate : 0.001
COMET INFO:   Uploads:
COMET INFO:     asset               : 2 (1.76 KB)
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: To get a